## Ablation Study

The **Ablation Study** script performs a low level comparison of the outputs of 2 search methods. Currently, it generates 5 graphs that can be used to assess and compare these 2 algorithms. The graphs show how each algorithm improves or underperforms the other, distribution of ranks that each algorithm returns a relevant document in, and other graphs that highlight how each model performs.

This script requires the prediction JSON files from 2 search algorithms and the ground truth JSON file. In our case, this can be done with our MSMarco subset with the Elasticsearch as the baseline result. An example of 2 model outputs to our MSMarco 100k subset and the ground truth is included under `./ablation_input`. You can run ablation study tool by running the the `ablation_study.py` script under `gamechangerml/src/search/evaluation` or calling the `AblationStudy` class from the script and run it in a Jupyter notebook.

For this example, we will be comparing 2 models: (1) **Base Elasticsearch** and (2) **the Relevance Retriever**. The Relevance Retriever algorithm adds words and phrases to the original text that a Transformer model sees as "relevant" in the context. More information about the Relevance Retriever can be found in [this link](https://confluence.di2e.net/display/UOT/Transformer+Enhanced+Keyword+Search).

To run the Ablation Study, you can use the command line script below.

```
python gamechangerml/src/search/evalution/ablation.py \
    -a <path/to/model/a/prediction> \
    -b <path/to/model/b/prediction> \
    -g <path/to/ground/truth> \
    -s <path/to/save/directory> \
    --model-a-name <Model A Name> \
    --model-b-name <Model B Name>
```

In [7]:
from gamechangerml.src.search.evaluation.ablation import AblationStudy

import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
study = AblationStudy(
    "./ablation_inputs/model_a_prediction.json",
    "./ablation_inputs/model_b_prediction.json",
    "./ablation_inputs/relations.json",
    results_path="./ablation_outputs",
    model_a_name = "Relevance Retriever",
    model_b_name = "Elasticsearch"
)

In [10]:
study.generate_report()
plt.close()

<Figure size 720x432 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 720x432 with 0 Axes>

<Figure size 576x576 with 0 Axes>

Matplotlib doesn't remove the plots so you can ignore the residual outputs above. Below, we show an analysis of what each graph looks like and how to interpret it.

### 1. Rank Occurence


<img src="./ablation_outputs/model_rank_occurence.png" alt="Drawing" style="width: 800px;"/>

The first graph generated is called the **Rank Occurence** graph. The graph plots out how many times each model returned a relevant document on a specific rank. Note that we're only noting the highest rank returned by the model. In the case that there are 2 relevant documents, we only consider the highest of the 2.

The graph answers the question "How many times did Model X return the relevant document at position Y?". We only look at the first 10 ranks as it is the rank that is most relevant to our analysis. This graph also shows a direct comparison of how many times Model A returns more documents at position X compared to Model B or vice versa.

### 2. Difference Distribution

<img src="./ablation_outputs/model_diff_dist.png" alt="Drawing" style="width: 800px;"/>

The next graph is called the **Difference Distribution** graph. The graph shows a more detailed look on how a model performs compared to the other. The graph is a heatmap where each pixel's value shows how many times the model score pair occurs. A model score pair is defined by the x and y coordinate of each pixel. The row indicates the score of Model B (in this case Elasticsearch) and the column indicates the score of Model A (in this case the Relevance Retriever). In this example, the pixel value at coordinate **(7, 4)** is **16**. This means there were 16 occurences where Elasticsearch ranked a relevant document at position 7 but the Relevance Retriever ranked it at 4. On a pixel level, it doesn't give a lot of insights. However, looking at it at a macroscopic level can provide some good understanding of their difference.

For example, notice that row 1 (Elasticsearch returned a relevant document at position 1) the Relevance Retriever at most reduces the score to position 5. However, the first column (Relevance Retriever returned a relevant document at position 2) shows occurences past position 5. This suggests that the new model is able to bring documents that the base Elasticsearch struggle to rank to high ranks.

Pixels that are further away from the diagonal show greater improvement of one model over the other.

### 3. Net Improvement

<img src="./ablation_outputs/net_improvement.png" alt="Drawing" style="width: 800px;"/>

Net improvement is simply counting the net score improvement between the 2 models. It first counts how many times Model A improves the score of Model B by 1 and vice versa. We then get the net value and plot it. Though it doesn't show where the improvement is, it shows how one model improves the score of the other.

### 4. Difference @ 10

<img src="./ablation_outputs/diff_10.png" alt="Drawing" style="width: 800px;"/>

The **Difference @ 10** graph shows how many times a model scored a certain value if it outperforms the other model.

### 5. Clear Improvement

<img src="./ablation_outputs/clear_improvement.png" alt="Drawing" style="width: 800px;"/>

The **Clear Improvement** graph is a histogram of points where a model ranks for queries where the other model doesn't return a relevant document. This use case is only important if our use case involves changing the Retriever model.